In [1]:
import pandas as pd 
import plotly.graph_objects as go
import plotly.express as px
import sys, os
fn_data = "../data/renewables_with_load.parquet"
from data import get_generation, normalize_generation, get_profiles
from graphs import plot_profile, plot_daily_generation

2024-02-12 12:34:29.518 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:29.519 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:29.521 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:29.522 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:29.523 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:29.524 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df_gen = get_generation(fn_data, country="DE", year=2023)
df_norm = normalize_generation(df_gen, shares={"Wind": 0.4, "Solar": 0.4, "Baseload": 0.2}, total_demand=0)
profiles = get_profiles(df_norm)

2024-02-12 12:34:29.824 
  command:

    streamlit run c:\GIT\baseload\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-02-12 12:34:29.825 No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:30.226 No runtime found, using MemoryCacheStorageManager
2024-02-12 12:34:30.232 No runtime found, using MemoryCacheStorageManager


In [14]:
def get_storage_stats(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Get curtailment and energy overshoot given the frame of generation

    Args:
        df: frame with hourly generation and demand data

    Returns:
        hourly dataframe with storage statistics; aggregated statistics
    """
    col_gen = [c for c in df.columns if c != "Demand"]
    df_ = df.assign(
        TotalSupply=lambda df: df[col_gen].sum(1),
        ExcessSupply=lambda df: (df["TotalSupply"] - df["Demand"]).map(
            lambda x: max(x, 0)
        ),
        ExcessDemand=lambda df: (df["Demand"] - df["TotalSupply"]).map(
            lambda x: max(x, 0)
        ),
    )
    total = df_.sum()
    stats = {
        var: {
            "Total": total[var],
            "TotalPercentOfDemand": total[var] / total["Demand"] * 100,
            "Max": df_[var].max(),
            "Hours": len(df_[df_[var] > 9]),
        }
        for var in ["ExcessSupply", "ExcessDemand"]
    }
    return df_, pd.DataFrame.from_dict(stats).round(1)

df, stats = get_storage_stats(df_norm)
stats
df_s = df.iloc[:24,:][["ExcessSupply", "ExcessDemand"]].copy()
df_s
stats


,ExcessSupply,ExcessDemand
Total,102985351.2,102985351.2
TotalPercentOfDemand,22.5,22.5
Max,104309.3,55468.4
Hours,3542.0,5215.0
